## Yandex SDC Meetup 2021 Contest

### Роботы-курьеры

https://contest.yandex.ru/contest/28643/problems/

Нужно написать упрощённую версию системы, управляющей роботами-доставщиками в Иннополисе: определить оптимальное количество роботов и построить для них маршруты.

### Описание задачи

Представим город в виде карты размера `N × N`.

Робот занимает ровно одну клетку и каждая клетка для него может быть либо проходимой, либо нет. \
За одну секунду робот может переместиться в любом из четырёх направлений \
(вверх/вниз/влево/вправо), если клетка, куда он хочет переместиться, свободна.

В начале теста нужно вывести количество роботов, которое решаем использовать для доставки заказов, \
и их изначальные координаты.

Постройка каждого робота будет стоить `Cost_c` рублей.

Далее будет произведено `T` итераций симуляции. \
Одна итерация представляет собой одну виртуальную минуту и состоит из 60 секунд. \
На каждой итерации вашей программе будет передано количество новых заказов, \
а в ответ программа должна сообщить, какие действия выполняет каждый робот \
(по 60 действий для робота).

За каждый успешно доставленный заказ получается `max(0, MaxTips - DeliveryTime)` рублей чаевых, где:

`MaxTips` — максимальное количество чаевых для одного заказа \
`DeliveryTime` — время с момента появления заказа до его доставки в секундах.

Итоговое количество очков, которое вы заработаете за один тест вычисляется по формуле \
`TotalTips - R × Costc`, где \
`TotalTips` — общее количество заработанных чаевых, \
`R` — количество использованных роботов, \
`Cost_c` — цена постройки одного робота. 

Значения `Cost_c` и `MaxTips` задаются в каждом тесте.

Если вы заработали меньше чаевых, чем потратили на производство роботов, итоговое количество очков будет равно 0. Также вы получите 0 очков за тест в случае выполнения любого некорректного действия.

### Формат ввода

Для чтения входных данных программа должна использовать стандартный ввод.

В первой строке ввода заданы три натуральных числа \
`N` — размер города, \
`MaxTips` — максимальное количество чаевых за заказ, \
`Cost_c` — цена постройки одного робота.

`(N ≤ 2 000, MaxTips ≤ 50 000, Cost_c ≤ 10 ** 9)`. 

Каждая из следующих `N` строк содержит `N` символов — карту города. \
Строки могут содержать два типа символов:

`'#'` - клетка занята препятствием. \
`'.'` - свободное пространство.

Затем на вход будет подано два целых натуральных числа `T` и `D` `(T ≤ 100 000, D ≤ 10 000 000)` — количество итераций взаимодействия и суммарное количество заказов.

Далее на каждой из `T` итераций мы сообщаем информацию о новых размещенных заказах. На каждой итерации сначала дано 
- целое число `k` — количество новых курьерских заказов, 
- затем `k` строк с числами `Srow`, `Scol`, `Frow`, `Fcol` — координаты начальной и конечной точки заказа \
`(1 ≤ Srow, Scol, Frow, Fcol ≤ N)`. 

Новый заказ может быть размещён в той же клетке, где уже находится 1 или более заказов. Время жизни заказа не ограничено.

После этого вам необходимо вывести число `R` - количество роботов, которые вы разместите в городе. Роботов должно быть не менее, чем 1 и не более, чем 100. Затем выведите `R` пар целых чисел от 1 до `N` — координаты, где роботы будут изначально расположены.

### Формат вывода

Для осуществления запросов программа должна использовать стандартный вывод.

На каждой итерации в ответ отправляется инфа о действиях каждого из роверов: \
`R` строк по 60 символов в каждой (один символ - одно действие, суммарно по 60 действий каждого ровера):

* `U` - движение на одну клетку вверх (уменьшить номер строки)
* `L` - движение на одну клетку влево (уменьшить номер столбца)
* `D` - движение на одну клетку вниз (увеличить номер строки)
* `R` - движение на одну клетку вправо (увеличить номер столбца)
* `S` - остаться на месте и ничего не делать
* `T` - остаться на месте и забрать самый старый заказ в текущей клетке
* `P` - остаться на месте и выдать заказ в текущей клетке

Роботы выполняют свои действия по очереди: 
- **сначала** первое действие выполняет первый робот, затем второй и так далее до последнего робота.
- **потом** первый робот выполняет второе действие, второй робот выполняет второе действие и т.д.
- **в конце** итерации каждый робот выполняет своё последнее действие и итерация заканчивается.

Несколько роботов могут занимать одну и ту же клетку. \
Робот не может перевозить более одного заказа одновременно.

Тестирующая система даст вашей программе прочитать свежие данные из входных данных только после того, как ваша программа вывела соответствующий запрос системе и выполнила операцию flush.

## SOLUTION

### Config

In [6433]:
import numpy as np
import heapq

In [6434]:
# Коды возможных действий ровера
MOVE  = 0
STAY  = 1
TAKE  = 2
PUT   = 3

# Коды возможных перемещений ровера
UP    = 10
DOWN  = 11
LEFT  = 12
RIGHT = 13


# Словарь действий ровера
ACTIONS_DICT = {
    UP   : {'sign': 'U', 'move': [-1,  0]},  # движение на одну клетку вверх (уменьшить номер строки)
    DOWN : {'sign': 'D', 'move': [ 1,  0]},  # движение на одну клетку вниз (увеличить номер строки) 
    LEFT : {'sign': 'L', 'move': [ 0, -1]},  # движение на одну клетку влево (уменьшить номер столбца)
    RIGHT: {'sign': 'R', 'move': [ 0,  1]},  # движение на одну клетку вправо (увеличить номер столбца)
    STAY : {'sign': 'S'},  # остаться на месте и ничего не делать
    TAKE : {'sign': 'T'},  # остаться на месте и забрать самый старый заказ в текущей клетке
    PUT  : {'sign': 'P'},  # остаться на месте и выдать заказ в текущей клетке
}

# Коды поисковых задач ровера
SEARCH_ORDER = 100
SEARCH_PATH = 101

# Коды возможных статусов заказа
PENDING     = 200
IN_PROGRESS = 201
DONE        = 202

ITER_DURATION = 60  # итерация состоит из 60 секунд

### Setup: Funcs and Classes

In [6435]:
def find_shortest_path(grid, begin, end):
    '''
    Функция для нахождения кратчайшего пути между двумя точками на матрице размерностью NxN
    '''
    N = len(grid)
    dist = np.zeros_like(grid)  # инициализируем матрицу расстояний в виде np.array NxN
    dist[begin[0], begin[1]] = 1  # устанавливаем начальную точку для матрицы расстояний равной 1
    q = [begin]  # задаем очередь для обработки точек
    # Перебираем в цикле пока очередь не закончится:
    while len(q):
        x, y = q.pop(0)  # извлекаем точку из очереди и берем её координаты
        if [x, y] == end:  # заканчиваем, если достигнута конечная точка
            return compose_shortest_path(N, dist, end)  # возвращаем кратчайший путь
        # Проверяем соседние точки по всем четырем направлениям:
        for xc, yc in ([max(x - 1, 0), y], [min(x + 1, N - 1), y], 
                       [x, max(y - 1, 0)], [x, min(y + 1, N - 1)]):
            if grid[xc, yc] == 0 and dist[xc, yc] == 0:
                dist[xc, yc] = dist[x, y] + 1
                q.append([xc, yc])
    return []

def compose_shortest_path(N, dist, end):
    '''
    Функция построения кратчайшего пути на основе матрицы расстояний
    '''
    path = []
    x, y = end  # начинаем с конечной точки
    while dist[x, y] != 1:  # движемся до тех пор, пока не упремся в начальную точку
        # Всегда передвигаемся к точке, имеющей текущее значение расстояния минус 1
        get_next_move = lambda x, y: {
            dist[max(x - 1, 0), y]: UP, 
            dist[min(x + 1, N - 1), y]: DOWN, 
            dist[x, max(y - 1, 0)]: LEFT, 
            dist[x, min(y + 1, N - 1)]: RIGHT 
        }[dist[x, y] - 1]

        move = get_next_move(x, y)
        x, y = np.add([x, y], ACTIONS_DICT[move]['move'])
        # Для прокладывания пути от начальной точки необходимо ревёрснуть направление:
        move = {
            UP: DOWN, 
            DOWN: UP, 
            LEFT: RIGHT, 
            RIGHT: LEFT
        }[move]
        path.append(move)
    return path[::-1]

In [6436]:
# https://gist.github.com/ryancollingwood/32446307e976a11a1185a5394d6657bc
class Node:
    '''A node class for A* Pathfinding'''
    def __init__(self, parent=None, pos=None):
        self.parent = parent
        self.pos = pos
        self.g = self.h = self.f = 0

    def __eq__(self, other):
        return self.pos == other.pos
    
    def __repr__(self):
        return f'{self.pos} - g: {self.g} h: {self.h} f: {self.f}'

    # defining less than for purposes of heap queue
    def __lt__(self, other):
        return self.f < other.f
    
    # defining greater than for purposes of heap queue
    def __gt__(self, other):
        return self.f > other.f

def return_path(current_node):
    path = []
    current = current_node
    while current is not None:
        path.append(current.pos)
        current = current.parent
        
    path = path[::-1]
        
    moves = [np.array(path[i + 1]) - path[i]
             for i in range(len(path) - 1)]

    get_next_move = lambda x, y: {
        x == -1: UP, 
        x ==  1: DOWN, 
        y == -1: LEFT,
        y ==  1: RIGHT, 
    }[True]

    return [get_next_move(*m) for m in moves]

def find_shortest_path(grid, begin, end):
    N = len(grid)
    start_node = Node(None, begin)
    end_node = Node(None, end)
    open_list = []
    closed_list = []
    
    heapq.heapify(open_list) 
    heapq.heappush(open_list, start_node)
    
    iterations = 0
    while len(open_list) > 0:
        iterations += 1
        if iterations > (N * N // 2):
#             print('Giving up on pathfinding: too many iterations!')
            return return_path(current_node)       
        
        current_node = heapq.heappop(open_list)
        closed_list.append(current_node)
        
        if current_node == end_node:
            return return_path(current_node)
        children = []
        
        # Проверяем соседние точки по всем четырем направлениям:
        for shift in ((0, -1), (0, 1), (-1, 0), (1, 0)): # adjacent squares            
            xc, yc = np.add(current_node.pos, shift)  # get node pos
            if not 0 <= xc < N or not 0 <= yc < N:  # make sure within range
                continue
            if grid[xc, yc] != 0:  # make sure walkable terrain
                continue
            # create new node and append to children dict
            children.append(Node(current_node, [xc, yc]))
            
        for ch in children:
            if len([closed_ch for closed_ch in closed_list 
                    if closed_ch == ch]) > 0:
                continue
            ch.g = current_node.g + 1
            ch.h = sum([(ch.pos[0] - end_node.pos[0]) ** 2,
                        (ch.pos[1] - end_node.pos[1]) ** 2])
            ch.f = ch.g + ch.h
            if len([open_node for open_node in open_list 
                    if ch.pos == open_node.pos and ch.g > open_node.g]) > 0:
                continue
            heapq.heappush(open_list, ch)
            
#     print("Couldn't get a path to destination")
    return None

In [6437]:
class Rover:
    def __init__(self, r_id, x, y, search_task=SEARCH_ORDER):
        self.r_id = r_id
        self.x = x
        self.y = y
        self.search_task = search_task
        self.o_id = None
        self.loaded = False
        self.earned = 0
        self.route = None
        self.next_action = None
        self.actions_str = ''
    
    def find_order(self, orders_pool, grid):
        orders_list = []
        for o_id, order in orders_pool.items():
            if order.status == PENDING:
                order.min_path = find_shortest_path(
                    grid, 
                    begin=[self.x, self.y], 
                    end=[order.x_from, order.y_from]
                )
                orders_list.append(o_id)

        if len(orders_list) == 0:
            self.next_action = STAY
            return None
        
        min_time = min([orders_pool[o_id].time_passed 
                        for o_id in orders_list])
        min_time_orders = [o_id for o_id in orders_list 
                           if orders_pool[o_id].time_passed == min_time]
        self.o_id = sorted(min_time_orders, 
                           key=lambda o_id: len(orders_pool[o_id].min_path), 
                           reverse=True)[0]
        self.route = orders_pool[self.o_id].min_path
        ### +++ DEBUG
        route = [ACTIONS_DICT[m]['sign'] for m in self.route]
        print(f'[>] R{self.r_id} to Ord{self.o_id} --> TAKEPATH: {route}')
        self.next_action = MOVE
        return orders_pool[self.o_id]
    
    def find_path(self, orders_pool, grid):
        x_end, y_end = 0, 0
        order = orders_pool[self.o_id]
        x_end, y_end = order.x_to, order.y_to
        ### +++ DEBUG
        print(f'Ord{self.o_id} destination: {x_end, y_end}; ', end='')
        self.route = find_shortest_path(
            grid, 
            begin=[self.x, self.y], 
            end=[x_end, y_end]
        )
        self.next_action = MOVE        
        ### +++ DEBUG
        route = [ACTIONS_DICT[m]['sign'] for m in self.route]
        print(f'route for R{self.r_id}: {route}')
    
    def move(self):
        ### +++ DEBUG
        route = [ACTIONS_DICT[m]['sign'] for m in self.route]
        print(f'R{self.r_id} MOVE ({route}):')
        current_move = ACTIONS_DICT[self.route.pop(0)]
        ### +++ DEBUG
        print(f'[{self.x}, {self.y}] -> ', end='')
        self.x, self.y = np.add([self.x, self.y], current_move['move'])
        ### +++ DEBUG
        print(f'[{self.x}, {self.y}]')
        self.actions_str += current_move['sign']
        if len(self.route) == 0:
            self.next_action = PUT if self.loaded else TAKE
            
    def stay(self):
        self.actions_str += ACTIONS_DICT[STAY]['sign']

    def take(self):
        ### +++ DEBUG
        print(f'R{self.r_id} TAKE')
        self.loaded = True
        self.search_task = SEARCH_PATH
        self.actions_str += ACTIONS_DICT[TAKE]['sign']
        
    def put(self, earned):
        ### +++ DEBUG
        print(f'R{self.r_id} PUT Ord{self.o_id}')
        if earned > 0: print(f'!! EARNED : {earned}')
        self.earned += earned
        self.o_id = None
        self.loaded = False        
        self.route = None
        self.search_task = SEARCH_ORDER
        self.actions_str += ACTIONS_DICT[PUT]['sign']

In [6438]:
class Order:
    def __init__(self, x_from, y_from, x_to, y_to, status):
        self.x_from = x_from
        self.y_from = y_from
        self.x_to = x_to
        self.y_to = y_to
        self.status = status
        self.time_passed = 0

### Input Factory

In [6439]:
def input_error(condition):
    return f'INPUT ERROR: Argument out of range ({condition})'

In [6440]:
def process_input_start(input_hook=None):
    '''В этой функции парсим поступающий стартовый ввод и извлекаем из него полезные данные'''
    if input_hook:  # определяем источник: std.input или список в памяти
        pixie = lambda: input_hook.pop(0)
    else:
        pixie = lambda: input()
    # Парсим pixie 
    (
        N,        # карта города имеет размер N * N
        MaxTips,  # максимальное количество чаевых для одного заказа
        Cost_c    # стоимость постройки каждого робота
    ) = [int(n) for n in pixie().split()]  # парсим начало input
    
    # Представляем карту города в виде матрицы { 0 - пусто; 1 - препятствие }
    CityMap = np.zeros((N, N), dtype=int)
    for i in range(N):
        row = [1 if ch == '#' else 0 for ch in pixie()]
        CityMap[i, :] = row
    
    (
        T,  # количество итераций взаимодействия
        D   # суммарное количество заказов
    ) = [int(n) for n in pixie().split()]  # парсим окончание input
    
    # Проверяем все переменные на соответствие диапазонам, заданным в условиях задачи
    for condition in ('N <= 2000', 'MaxTips <= 50000', 'Cost_c <= 10**9', 
                      'T <= 100000', 'D <= 10000000'):
        assert eval(condition), input_error(condition)
    return N, CityMap, MaxTips, Cost_c, T, D

In [6441]:
def process_input_orders(N, input_hook=None):
    '''Функция для парсинга информации о новых поступающих заказах'''
    if input_hook:  # определяем источник: std.input или список в памяти
        pixie = lambda: input_hook.pop(0)
    else:
        pixie = lambda: input()
    # Парсим pixie
    k = int(pixie())  # количество новых курьерских заказов
    
    new_orders = []    
    for i in range(k):
        # координаты начальной и конечной точки заказа
        S_row, S_col, F_row, F_col = [int(c) for c in pixie().split()]
        print(f'Ord{i}:', S_row, S_col, '->', F_row, F_col)

        # проверяем координаты на соответствие: (1 ≤ S_row, S_col, F_row, F_col ≤ N)
        for condition in [f'1 <= {coord} <= N' 
                          for coord in ('S_row', 'S_col', 'F_row', 'F_col')]:
            assert eval(condition), input_error(condition)

        # адаптируем к внутренней системе координат и добавляем в список новых заказов
        new_orders.append(list(map(lambda x: x - 1, 
                                   (S_row, S_col, F_row, F_col))))
    return new_orders

In [6442]:
def random_orders(N, grid, kmax=5):
    k = np.random.randint(kmax)
    orders = []
    obstacles = [(x, y) for x in range(N) for y in range(N) if grid[x, y] == 1]
    for i in range(k):
        ok = False
        while not ok:
            start_coords  = (np.random.randint(1, N + 1), 
                             np.random.randint(1, N + 1))
            finish_coords = (np.random.randint(1, N + 1), 
                             np.random.randint(1, N + 1))
            if (start_coords not in obstacles and 
                finish_coords not in obstacles and
                start_coords != finish_coords):
                ok = True
        S_row, S_col, F_row, F_col = *start_coords, *finish_coords
        orders += [f'{S_row} {S_col} {F_row} {F_col}']
    return [str(k)] + orders

### Simulation

In [6443]:
class Simulation:
    def run(self, input_hook=None):
        (self.N, 
         self.grid, 
         self.MaxTips, 
         self.Cost_c, 
         self.T, 
         self.D) = process_input_start(input_hook)
        (self.R, 
         self.rovers_pool) = self.decide_start()
        
        return_tips = True if input_hook else False
        total_tips = self.run_iterations(input_hook, return_tips=return_tips)
        if total_tips:
            self.results(total_tips)
        
    def decide_start(self):
        free_cells = np.argwhere(self.grid == 0)  # координаты всех пустых клеток
        # Для обеспечения доставки всех заказов минимальное число роверов 
        # считаем как: D {общее количество заказов} / T {количество итераций}:
        R = max(1, min(round(self.D / self.T), 100))  # количество размещаемых роверов (1 ≤ R ≤ 100)
        # Определим начальное расположение всех роверов
        rovers_pool = []  # пул роверов
        for r_id in range(R):
            # располагаем радномно в свободных клетках:
            x, y = free_cells[np.random.choice(len(free_cells))]
            rovers_pool.append(
                Rover(r_id, x, y, search_task=SEARCH_ORDER)  # спауним новый ровер
            )
        # Передадим службе ответную информацию
        print(R, flush=True)  # количество роверов
        for r in rovers_pool:
            print(r.x + 1, r.y + 1, flush=True)  # координаты - числа от 1 до N
        return R, rovers_pool

    def run_iterations(self, input_hook=None, return_tips=False):
        self.orders_pool = {}
        last_o_id = 0
        # Производим T итераций
        for it in range(self.T):
            print(f' ITERATION #{it + 1} '.center(60, '='))
            # На каждой итерации получаем информацию о новых размещенных заказах
            new_orders = process_input_orders(self.N, input_hook)  # считываем новые заказы
            if len(new_orders) > 0:
                for o_id, o_coords in enumerate(new_orders):
                    self.orders_pool[last_o_id + o_id] = Order(
                        *o_coords, status=PENDING,  # спауним новый заказ
                    )
                last_o_id += len(new_orders)

            response = self.iteration(self.R, self.rovers_pool, self.orders_pool)

            # На каждой итерации в ответ возвращаем по 60 действий для каждого из роверов
            for actions in response:
                print(actions, flush=True)
        if return_tips:
            return sum([rover.earned for rover in self.rovers_pool])
        
    def iteration(self, R, rovers_pool, orders_pool):
        '''Механика итерации'''
        iter_actions = []
        for iter_time in range(ITER_DURATION):
            # Сначала обрабатываем все поисковые задачи роверов (поиск заказов, поиск маршрута)
            for r_id in range(R):
                self.search_logic(rovers_pool[r_id])
            # Далее обрабатываем действия роверов (движение, получение и отгрузка заказа)
            for r_id in range(R):
                self.actions_logic(rovers_pool[r_id])
            # Добавляем секунду к времени существования всех заказов
            for order in orders_pool.values():
                order.time_passed += 1
        # Формируем и возвращаем список действий всех роверов в текущей итерации
        for r_id in range(R):
            iter_actions.append(rovers_pool[r_id].actions_str)
            rovers_pool[r_id].actions_str = ''
        return iter_actions

    def search_logic(self, rover):
        '''Поисковая логика роверов'''
        # Подключаем свободные роверы к обработке ближайших ожидающих заказов
        if rover.search_task == SEARCH_ORDER:
            order = rover.find_order(self.orders_pool, self.grid)
            if order is not None:
                order.status = IN_PROGRESS
                rover.search_task = None
        # Прокладываем кратчайший маршрут
        if rover.search_task == SEARCH_PATH:
            rover.find_path(self.orders_pool, self.grid)
            rover.search_task = None

    def actions_logic(self, rover):
        '''Логика действий роверов'''
        if rover.next_action == MOVE:
            rover.move()
        elif rover.next_action == STAY:
            rover.stay()
        elif rover.next_action == TAKE:
            rover.take()
        elif rover.next_action == PUT:
            order = self.orders_pool[rover.o_id]
            order.status = DONE
            earned = max(0, self.MaxTips - order.time_passed)
            rover.put(earned)
            
    def results(self, total_tips):
        # Итоговое количество очков, заработанное за один тест
        result = total_tips - self.R * self.Cost_c
        print(f'Total tips: {total_tips}; Total build cost: {self.R * self.Cost_c}')
        print(f'Result: {result}')

### Run

In [6444]:
def read_test(file='01', test_dir='examples\\'):
    with open(test_dir + file) as f:
        return [row.strip() for row in f]

input_hook = read_test('01')
# input_hook = None

In [6445]:
%%time

sim = Simulation()
sim.run(input_hook)

1
4 3
======================= ITERATION #1 =======================
Ord0: 1 1 -> 4 4
[>] R0 to Ord0 --> TAKEPATH: ['U', 'L', 'U', 'L', 'U']
R0 MOVE (['U', 'L', 'U', 'L', 'U']):
[3, 2] -> [2, 2]
R0 MOVE (['L', 'U', 'L', 'U']):
[2, 2] -> [2, 1]
R0 MOVE (['U', 'L', 'U']):
[2, 1] -> [1, 1]
R0 MOVE (['L', 'U']):
[1, 1] -> [1, 0]
R0 MOVE (['U']):
[1, 0] -> [0, 0]
R0 TAKE
Ord0 destination: (3, 3); route for R0: ['R', 'D', 'R', 'D', 'R', 'D']
R0 MOVE (['R', 'D', 'R', 'D', 'R', 'D']):
[0, 0] -> [0, 1]
R0 MOVE (['D', 'R', 'D', 'R', 'D']):
[0, 1] -> [1, 1]
R0 MOVE (['R', 'D', 'R', 'D']):
[1, 1] -> [1, 2]
R0 MOVE (['D', 'R', 'D']):
[1, 2] -> [2, 2]
R0 MOVE (['R', 'D']):
[2, 2] -> [2, 3]
R0 MOVE (['D']):
[2, 3] -> [3, 3]
R0 PUT Ord0
!! EARNED : 8
ULULUTRDRDRDPSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS

======================= ITERATION #2 =======================
Ord0: 1 4 -> 4 1
[>] R0 to Ord1 --> TAKEPATH: ['U', 'U', 'U']
R0 MOVE (['U', 'U', 'U']):
[3, 3] -> [2, 3]
R0 MOVE (['U', 'U']):
[2, 3]